In [21]:
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import torchvision
import torch
import os
import random
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, roc_curve, plot_roc_curve, auc
from sklearn.model_selection import train_test_split
from graspologic.cluster import GaussianCluster as GMM
from collections import defaultdict
from proglearn.forest import UncertaintyForest


In [2]:
acorn = 1234
torch.manual_seed(acorn)
np.random.seed(acorn)

torch.cuda.is_available()

if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
    
device = torch.device(dev)  

n_iter = 10
seeds = np.random.randint(10000, size=n_iter)

In [3]:
#process data, filter out only frontal, ap, fillter out uncertainty in classes we care and fill in rest data
def process_data(df):
    
    print('starting size %s' %len(df))
    data = df
    #only use frontal/AP data
    data = data.loc[data['Frontal/Lateral'] == 'Frontal']
    data = data.loc[data['AP/PA'] == 'AP']

    
    category_names = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']
    
    #filter out all uncertainty labels in classes we care about
    data = data[category_names]
    #tread all empty values in these selected cols as 0
    data = data.fillna(0)
    #filter out -1 (uncertain labels)
    data = data.loc[(data.iloc[:, :] !=-1).all(axis=1)]
    #row-idx of the data we care to keep
    fly_list = data.index
    #reselect from orginal of kept rows
    data = df.iloc[fly_list]

    #select the cols we care about
    wanted_cols = ["Path", 'No Finding'] + category_names
    data = data[wanted_cols]
    
    #filter out rows with no label values
    data['sum']  = data.iloc[:, 1:].sum(axis=1)
    fly_list = data.loc[data['sum']>0].index

    
    data = df[wanted_cols].iloc[fly_list]
    # fill all NA and uncertainty as 0     
    data = data.fillna(0)
    data = data.replace(-1,0)

    print("final size %s" %len(data))
    return data



In [4]:
data_root = '/home/weiwya/teamdrive_bak/weiwei_temp_data'
test_df = pd.read_csv('%s/CheXpert-v1.0-small/valid.csv' %data_root)
test_df = process_data(test_df)

train_full = pd.read_csv('%s/CheXpert-v1.0-small/train.csv' %data_root)
train_full = process_data(train_full)

train_full.head()

starting size 234
final size 132
starting size 223414
final size 92771


,Path,No Finding,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
0,CheXpert-v1.0-small/train/patient00001/study1/...,1.0,0.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0-small/train/patient00003/study1/...,0.0,0.0,0.0,0.0,1.0,0.0
11,CheXpert-v1.0-small/train/patient00006/study1/...,1.0,0.0,0.0,0.0,0.0,0.0
12,CheXpert-v1.0-small/train/patient00007/study1/...,0.0,1.0,1.0,0.0,0.0,0.0
13,CheXpert-v1.0-small/train/patient00007/study2/...,0.0,1.0,0.0,0.0,0.0,0.0


In [5]:
class ChestXRayDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform, data_root):
        #TODO::put something here that perserves aspect ratio
        self.class_names = ['No Finding', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']
        self.image_dir = data_root
        self.transform = transform
        self.total = len(df)
        self.image_names = df['Path'].to_list()
        self.labels = df[self.class_names].to_numpy()
                    
    def __len__(self):
        return self.total
    
    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = self.transform(Image.open(image_path).convert('RGB'))
        label = self.labels[idx]
        return image, label

    
image_size = (320, 320)
resnet_mean = [0.485, 0.456, 0.406]
resnet_std = [0.229, 0.224, 0.225]

#Creating a Transformation Object
train_transform = torchvision.transforms.Compose([
    #Converting images to the size that the model expects
    torchvision.transforms.Resize(size=image_size),
    torchvision.transforms.RandomHorizontalFlip(), #A RandomHorizontalFlip to augment our data
    torchvision.transforms.ToTensor(), #Converting to tensor
    #Normalizing the data to the data that the ResNet18 was trained on
    torchvision.transforms.Normalize(mean = resnet_mean ,
                                    std = resnet_std) 
    
])


#Creating a Transformation Object
test_transform = torchvision.transforms.Compose([
    #Converting images to the size that the model expects
    torchvision.transforms.Resize(size=image_size),
    # We don't do data augmentation in the test/val set    
    torchvision.transforms.ToTensor(), #Converting to tensor
    torchvision.transforms.Normalize(mean = resnet_mean,
                                    std = resnet_std) 
    
])


In [6]:
#take any model, use its penultimate layer as output features
def extract_fetures_targets(model, dl):    
    tt = model.base_model
    modules=list(tt.children())[:-1]
    feature_extractor = torch.nn.Sequential(*modules)
    for p in feature_extractor.parameters():
        p.requires_grad = False

    feature_extractor.to(device)
    features = []
    targets = []
    for val_step, (images, labels) in enumerate(dl):
        imagesGPU = images.to(device)      
        outputs = feature_extractor(imagesGPU)        
        outputs = torch.Tensor.cpu(outputs)
        outputs = outputs.detach().numpy()
        features.append(outputs)
        targets.append(labels)
        
    features = np.vstack(features)
    targets = np.vstack(targets)
    dim = features.shape[1]
    features= features.reshape(len(dl.dataset), dim)
    torch.cuda.empty_cache()
    return features, targets


#build model using Resnet50 as backbone
class Resnext50(torch.nn.Module):
    def __init__(self, n_classes, name):
        super().__init__()
        resnet = torchvision.models.resnet18(pretrained=True)
        resnet.fc = torch.nn.Sequential(
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(in_features=resnet.fc.in_features, out_features=n_classes)
        )
        self.base_model = resnet
        self.sigm = torch.nn.Sigmoid()
        self.name = name

    def forward(self, x):
        return self.sigm(self.base_model(x))

def eval_model (model, loss_fn, dl, verbose=True):
    model.eval()
    predicts = []
    targets = []
    total_loss = 0
    class_lookup = dl.dataset.class_names
    n_class = len(class_lookup)

    for val_step, (images, labels) in enumerate(dl):

        imagesGPU, labelsGPU = images.to(device), labels.to(device)        
        outputs = model(imagesGPU)
        loss = loss_fn(outputs, labelsGPU.type(torch.float))       
        total_loss += loss.item()               
        outputs = torch.Tensor.cpu(outputs)
        predicts.append(outputs.detach().numpy())
        targets.append(labels)

    predicts = np.vstack(predicts)
    targets = np.vstack(targets)
    loss = total_loss/len(dl)
    
    res = {}
    total_auc = 0
    total_counts = 0
    for idx in range(n_class):
        truth  = targets[:, idx]
        pp = predicts[:,idx]
        fpr, tpr, thresholds = roc_curve(truth, pp)
        auc_score = auc(fpr, tpr)
        res[class_lookup[idx]] = auc_score
        counts = np.sum(truth)
        total_auc += auc_score * counts
        total_counts += counts
    avg_auc = total_auc / total_counts
    
    if verbose:
        for k, v in res.items():
            print(k, v)
        print()
        print('loss:%s, avg_auc:%s' %(loss, avg_auc))
        
    return avg_auc, loss

def eval_fine_model(model, loss_fn, dl, verbose=True):
    model.eval()
    predicts = []
    targets = []
    total_loss = 0
    class_lookup = dl.dataset.class_names

    with tqdm(dl, unit="batch") as tepoch:
        for images, labels in tepoch:
            imagesGPU, labelsGPU = images.to(device), labels.to(device)        
            outputs = model(imagesGPU)
            loss = loss_fn(outputs, labelsGPU.type(torch.float))       
            total_loss += loss.item()               
            outputs = torch.Tensor.cpu(outputs)
            predicts.append(outputs.detach().numpy())
            targets.append(labels)
            tepoch.set_postfix(loss=loss.item())

    predicts = np.vstack(predicts)
    targets = np.vstack(targets)
    loss = total_loss/len(dl)
    
    n_samples = len(targets)
    n_actual_classes = len(dl.dataset.class_names)
    n_fine_classes = targets.shape[1]
    
    
    lookup = dl.dataset.fine_to_org
    coarse_predict = np.zeros((n_samples, n_actual_classes))
    coarse_target = np.zeros((n_samples, n_actual_classes))
    
    res = {}
    total_auc = 0
    total_counts = 0
    class_lookup = dl.dataset.class_names
    
    #add all fine class predictions to their proper coarse label
    for idx in range(n_fine_classes):
        coarse_idx = lookup[idx]
        coarse_predict[:, coarse_idx] += predicts[:, idx]
        coarse_target[:, coarse_idx] += targets[:, idx]
    
    
    for idx in range(n_actual_classes): 
        truth  = coarse_target[:, idx]        
        pp = coarse_predict[:,idx]
        
        truth = np.clip(truth, 0, 1.0)
        pp = np.clip(pp, 0, 1.0)
        
        fpr, tpr, thresholds = roc_curve(truth, pp)
        auc_score = auc(fpr, tpr)
        res[class_lookup[idx]] = auc_score
        counts = np.sum(truth)
        total_auc += auc_score * counts
        total_counts += counts
    
    avg_auc = total_auc / total_counts
    
    if verbose:
        for k, v in res.items():
            print(k, v)
        print()
        print('loss:%s, avg_auc:%s' %(loss, avg_auc))
        
    return avg_auc, loss

def train_model(epochs, train_model, train_loss_fn, train_optimizer, dl_train, dl_validate, dl_test, eval_fn=None):
    for e in range(0, epochs):
        train_loss = 0.        
        train_model.train() 
        with tqdm(dl_train, unit="batch") as tepoch:
            for images, labels in tepoch:
                images, targets = images.to(device), labels.to(device)
                train_optimizer.zero_grad()
                outputs = train_model(images)
                loss = train_loss_fn(outputs, targets.type(torch.float))

                #Once we get the loss we need to take a gradient step
                loss.backward() #Back propogation
                train_optimizer.step() #Completes the gradient step by updating all the parameter values(We are using all parameters)
                train_loss += loss.item() #Loss is a tensor which can't be added to train_loss so .item() converts it to float                
                tepoch.set_postfix(loss=loss.item())
        
        print('train_loss %s ' %(train_loss / len(dl_train)))
        if eval_fn is not None:
            if dl_validate is not None:
                eval_fn(train_model, dl_validate)
            if dl_test is not None:
                eval_fn(train_model, dl_test)
                
        print('done %s' %e)
    print('Training complete..')

In [7]:
#turn a item:list(classes) into a 1-hot-labels
def make_vector_label(lookup, n_classes):
    n_samples = len(lookup)
    labels = np.zeros( (n_samples, n_classes))
    for idx, v in lookup.items():
        for vv in v:
            labels[idx][vv] = 1.0
    return labels

#1st cluster within a label
#then cluster means of each cluster to generate coarse label
#return new label in for each feature in coarse, fine and also a fine to orignal lookup
def gen_coarse_fine_labels(features, labels, n_cluster_min=5, n_cluster_max=8):
    fine_clfs = []
    conditional_means = []
    n_features = labels.shape[1]
    idx_to_fine_labels = defaultdict(list)
    curr = 0
    fine_to_org ={}
    
    for i in range(n_features):
        ll = labels[:, i]
        selected_idx = np.where(ll==1.0)[0]
        xx = features[selected_idx]
        
        
        clf = GMM(min_components=n_cluster_min, max_components=n_cluster_max, reg_covar=1e-3).fit(xx) 
        pp = clf.predict(xx) + curr
        curr += clf.n_components_

        unique_y = np.unique(pp)
        for y in unique_y:
            fine_to_org[y] = i
            
        means = np.array([
            np.mean(features[np.where(pp == c)[0]], axis=0) for c in unique_y])
        conditional_means.append(means)
        fine_clfs.append(clf)
        
        #add fine labels to item lookup
        for idx, p in zip(selected_idx, pp):
            idx_to_fine_labels[idx].append(p)
            
    conditional_means = np.vstack(conditional_means)
    total_fine_labels = conditional_means.shape[0]

    coarse_clf  = GMM(min_components=n_cluster_min, max_components=n_cluster_max, reg_covar=1e-3)
    coarse_clf.fit(conditional_means)
    total_coarse_labels = coarse_clf.n_components_
    
    pc = coarse_clf.predict(conditional_means)
    
    #lookup for which coarse cluster a fine lable belongs
    
    fine_to_coarse = {}
    for idx, p in enumerate(pc):
        fine_to_coarse[idx] = p

    idx_to_coarse_labels = defaultdict(set)

    for idx, fine_label in idx_to_fine_labels.items():
        for f in fine_label:
            coarse_label = fine_to_coarse[f]
            idx_to_coarse_labels[idx].add(coarse_label)
        
    return (idx_to_coarse_labels, total_coarse_labels), (idx_to_fine_labels, total_fine_labels), fine_to_org   
    
        
        

In [8]:
#take any model, use its penultimate layer as output features
def get_feature_extractor(model):    
    tt = model.base_model
    modules=list(tt.children())[:-1]
    feature_extractor = torch.nn.Sequential(*modules)
    for p in feature_extractor.parameters():
        p.requires_grad = False
    feature_extractor.to(device)
    return feature_extractor


def gen_fine_clf(features, labels, n_cluster_min=5, n_cluster_max=8):
    fine_clfs = []
    n_features = labels.shape[1]
    curr = 0
    fine_to_org ={}
    
    for i in range(n_features):
        ll = labels[:, i]
        selected_idx = np.where(ll==1.0)[0]
        xx = features[selected_idx]
           
        clf = GMM(min_components=n_cluster_min, max_components=n_cluster_max, reg_covar=1e-3).fit(xx) 
        pp = clf.predict(xx) + curr
        curr += clf.n_components_

        unique_y = np.unique(pp)
        for y in unique_y:
            fine_to_org[y] = i
            
        fine_clfs.append(clf)
        
    return fine_clfs, fine_to_org, curr


def make_fine_labels(features, org_labels, fine_clfs, n_fine_classes):

    fine_labels = np.zeros(n_fine_classes)
    curr = 0
    for i in range(org_labels.shape[1]):
        ll = org_labels[0, i]
        fine_clf = fine_clfs[i]
        if ll == 1:
            pp = fine_clf.predict(features) + curr
            fine_labels[pp[0]] = 1.
        curr += fine_clf.n_components_
        
    return fine_labels
        
        
    
def extract_features(feature_extractor, tensor):
    (channel, w, h ) = tensor.shape

    tt = tensor.reshape(1, channel, w, h)
    tensor_gpu = tt.to(device)
    outputs = feature_extractor(tensor_gpu)        
    outputs = torch.Tensor.cpu(outputs)
    outputs = outputs.detach().numpy()
    dim = outputs.shape[1]
    outputs = outputs.reshape(1, dim)
    return outputs
    

#load data, and convert into features to inferr finelabels
class ChestXRayFineLabelDataset(torch.utils.data.Dataset):
    def __init__(self, df, transform, data_root, embd_model, fine_clfs, fine_to_org, n_fine_classes):
        self.class_names = ['No Finding', 'Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Pleural Effusion']
        self.image_dir = data_root
        self.transform = transform
        self.total = len(df)
        self.image_names = df['Path'].to_list()
        self.org_labels = df[self.class_names].to_numpy()
        self.embd_model = embd_model
        self.fine_clfs = fine_clfs
        self.fine_to_org = fine_to_org
        self.n_fine_classes = n_fine_classes
                    
    def __len__(self):
        return self.total
    
    def __getitem__(self, idx):
        image_name = self.image_names[idx]
        image_path = os.path.join(self.image_dir, image_name)
        image = self.transform(Image.open(image_path).convert('RGB'))
        features = extract_features(self.embd_model, image)
        org_label = self.org_labels[idx]
        org_label = org_label.reshape(1, -1)
        fine_labels = make_fine_labels(features, org_label, self.fine_clfs, self.n_fine_classes)
        return image, fine_labels

 

In [16]:
train_size = 0.1
#generate n_iter times of train/validate split
trains, validates = [],[]
for i in range(n_iter):
    train, validate = train_test_split(train_full, test_size=1-train_size, random_state=seeds[i], shuffle=True)
    trains.append(train)
    validates.append(validate)
    
    
batch_size = 16
coarse_models = []
fine_models = [] 

base_test_auc = []
base_validate_auc = []
ours_test_auc = []
ours_validate_auc = []

train_epoch= 3

for iteration in range(n_iter):
    
    train_df, validate_df = trains[iteration], validates[iteration]
    train_dataset = ChestXRayDataset(train_df, train_transform, data_root)
    valid_dataset = ChestXRayDataset(validate_df, test_transform, data_root)
    test_dataset = ChestXRayDataset(test_df, test_transform, data_root)

    batch_size = 16

    dl_train = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    dl_validate = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
    dl_test = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
    
    print('iteration: %i train_size: %i, validate_size: %i, test_size: %i' 
              %(iteration, len(dl_train), len(dl_validate), len(dl_test) ))
    
    # Initialize the model
    c_model = Resnext50(len(train_dataset.class_names), 'coarse_nn')
    c_model.to(device)
    c_loss_fn = torch.nn.BCELoss().to(device)
    c_optimizer = torch.optim.Adam(c_model.parameters(), lr=5e-5)
    
    
    train_model(train_epoch, c_model, c_loss_fn, c_optimizer, dl_train, dl_validate, dl_test, eval_fn=None)
    coarse_models.append(c_model)
    print('done training embedding NN')
    
    #these will be the base we measure improvements
    valid_auc, _ = eval_model(c_model, c_loss_fn, dl_validate)
    test_auc, _  = eval_model(c_model, c_loss_fn, dl_test)
    base_validate_auc.append(valid_auc)
    base_test_auc.append(test_auc)
    print('done eval base model %s %s' %(valid_auc, test_auc))
    
    #do our thingy
    embd_model = get_feature_extractor(c_model)
    train_features, train_targets = extract_fetures_targets(c_model, dl_train)
    fine_clfs, fine_to_org, n_fine_classes = gen_fine_clf(train_features, train_targets)

    train_fine_dataset = ChestXRayFineLabelDataset(train_df, train_transform, data_root, embd_model, fine_clfs, fine_to_org, n_fine_classes)
    test_fine_dataset = ChestXRayFineLabelDataset (test_df, test_transform, data_root, embd_model, fine_clfs, fine_to_org, n_fine_classes)
    validate_fine_dataset = ChestXRayFineLabelDataset (validate_df, test_transform, data_root, embd_model, fine_clfs, fine_to_org, n_fine_classes)
    
                                                       
    dl_fine_train = torch.utils.data.DataLoader(train_fine_dataset, batch_size=batch_size, shuffle=True)
    dl_fine_test = torch.utils.data.DataLoader(test_fine_dataset, batch_size=batch_size, shuffle=True)
    dl_fine_valid = torch.utils.data.DataLoader(validate_fine_dataset, batch_size=batch_size, shuffle=True)
                                                       
                                                       
    fine_model = Resnext50(n_fine_classes, 'fine_model')
    fine_model.to(device)
    fine_loss_fn = torch.nn.BCELoss().to(device)
    fine_loss_fn.name="fine_lost"
    fine_optimizer = torch.optim.Adam(fine_model.parameters(), lr=5e-5)
    train_model(train_epoch, fine_model, fine_loss_fn, fine_optimizer, dl_fine_train, dl_validate=None, dl_test=None, eval_fn=None)
    fine_models.append(fine_model)
                                                       
    #these will be the base we measure improvements
    our_valid_auc, _ = eval_fine_model(fine_model, fine_loss_fn, dl_fine_valid)
    our_test_auc, _  = eval_fine_model(fine_model, fine_loss_fn, dl_fine_test)
    ours_validate_auc.append(our_valid_auc)
    ours_test_auc.append(our_test_auc)
    print('done-iterations %s\n val: %s test: %s\n our_val:%s our_test %s' 
          %(iteration, valid_auc, test_auc, our_valid_auc, ours_test_auc))
             
    

iteration: 0 train_size: 580, validate_size: 5219, test_size: 9


  0%|          | 1/580 [00:00<01:02,  9.23batch/s, loss=0.457]

train_loss 0.4504555628731333 
done 0


  0%|          | 1/580 [00:00<01:05,  8.78batch/s, loss=0.466]

train_loss 0.3972012645211713 
done 1


...

train_loss 0.37189876432048863 
done 2
Training complete..
done training embedding NN
No Finding 0.8781969484323571
Atelectasis 0.6410196746102139
Cardiomegaly 0.807075313094683
Consolidation 0.6899493543244206
Edema 0.7789961404686648
Pleural Effusion 0.8421822056750534

loss:0.41146207949877195, avg_auc:0.7860148126212773
No Finding 0.9618055555555556
Atelectasis 0.6305703070884322
Cardiomegaly 0.7848450057405282
Consolidation 0.8034375
Edema 0.860895202358617
Pleural Effusion 0.8688524590163935

loss:0.6420648429128859, avg_auc:0.7848715856020674
done eval base model 0.7860148126212773 0.7848715856020674


  0%|          | 0/580 [00:00<?, ?batch/s]

train_loss 0.20070785420208143 
done 0


  4%|▍         | 25/580 [00:07<02:45,  3.35batch/s, loss=0.129]


KeyboardInterrupt: 

In [17]:
coarse_stuff, fine_stuff, lookup_dic = gen_coarse_fine_labels(train_features, train_targets)

In [26]:
test_features, test_targets = extract_fetures_targets(c_model, dl_test)

In [ ]:
class_wise_clfs = []
aucs = []
counts = 0
for idx in range(6):
    target = train_targets[:, idx]
    clf = UncertaintyForest(n_estimators=100, max_depth=10)
    clf.fit(train_features, target)
    pp = clf.predict(train_features)
    fpr, tpr, thresholds = roc_curve(target, pp)
    auc_score = auc(fpr, tpr)
    if idx != 0:
        aucs.append(auc_score * sum(target))
        counts+= sum(target)
    class_wise_clfs.append(clf)
    print (train_dataset.class_names[idx], auc_score)

No Finding 0.9088632874866905
Atelectasis 0.8317360268701492


In [30]:
aucs = []
counts = 0
for i in range(6):
    target = test_targets[:, i]
    pp = class_wise_clfs[i].predict(test_features)
    fpr, tpr, thresholds = roc_curve(target, pp)
    auc_score = auc(fpr, tpr)
    if idx != 0:
        aucs.append(auc_score * sum(target))
        counts+= sum(target)
    
    print (train_dataset.class_names[i], auc_score) 
print(sum(aucs)/counts)

Pleural Effusion 0.8041666666666666
Pleural Effusion 0.5517201570076196
Pleural Effusion 0.5391504018369689
Pleural Effusion 0.5625
Pleural Effusion 0.7736531760922004
Pleural Effusion 0.7338951743246364
0.6324620323422943


In [14]:
test_auc

0.7932935296733508